In [6]:
%load_ext autoreload
%autoreload 2

import os
import sys

import requests
from bs4 import BeautifulSoup as bs
from bs4.element import Tag, NavigableString

project_path = os.path.abspath("..")

sys.path.append(project_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from src.scraper import (
    download_unit_icon,
    extract_block_data,
    extract_item_vals,
    extract_unit_data,
)

In [8]:
def fetch_page_content(url):
    """Fetches the HTML content of the page."""
    response = requests.get(url)
    response.raise_for_status()
    # Raises an error for bad responses (e.g., 404, 500)
    return response.text

In [9]:
url = "https://ageofempires.fandom.com/wiki/War_Wagon_(Age_of_Empires_III)"
html_content = fetch_page_content(url)
soup = bs(html_content, "html.parser")
infobox = soup.find('aside', class_='portable-infobox')

In [7]:
img_path = os.path.join(project_path, "images")
download_unit_icon(infobox, img_path)

Icono guardado como: 'war_wagon_de.png' en 'c:\Users\Jonathan Diaz\Documentos\Proyects\aoe3de_web_scraper\images'


In [9]:
blocks = infobox.find_all("section")
block = blocks[0]
extract_block_data(block)

label:  Introduced in
value_type:  str
values:  Age of Empires III

label:  Type
value_type:  list
values:  ['Cavalry', 'Light ranged cavalry', 'Ranged cavalry', 'Gunpowder cavalry']

label:  Civilization(s)
value_type:  list
values:  ['Germans', "John Black's Mercenaries (Act II: Ice)", 'South Africa']

label:  Age
value_type:  list
values:  ['Fortress Age', 'Revolution (South Africa)']

label:  Required Home City Card
value_type:  str
values:  "Wagon Warfare" (South Africa)



{'Introduced in': 'Age of Empires III',
 'Type': ['Cavalry',
  'Light ranged cavalry',
  'Ranged cavalry',
  'Gunpowder cavalry'],
 'Civilization(s)': ['Germans',
  "John Black's Mercenaries (Act II: Ice)",
  'South Africa'],
 'Age': ['Fortress Age', 'Revolution (South Africa)'],
 'Required Home City Card': '"Wagon Warfare" (South Africa)'}

In [10]:
statistic_labels = [
    "Cost",
    "Resistance",
    "Damage",
    "Bonus damage",
]

rows = block.find_all("div", class_="pi-item", recursive=False)
row = rows[0]
label = row.find("h3").text
is_st_label = label in statistic_labels

elements = row.find("div", class_="pi-data-value")

values = extract_feat_values(elements, is_st_label)

print("label: ", label)
print("value_type: ", type(values))
print("values: ", values)

label:  Introduced in
value_type:  <class 'str'>
values:  Age of Empires III


In [134]:
block = blocks[2]
rows = block.find_all("div", class_="pi-item", recursive=False)
row = rows[1]
label = row.find("h3").text
item = row.find("div", class_="pi-data-value")
for feat in item.children:
    if isinstance(feat, Tag) and "image" in (feat.get("class") or []):
        continue
    
    print("=========================================================================================")
    print("=========================================================================================")
    print("Type: ", type(feat).__name__)
    print("Name Tag: ", feat.name)
    # print("Class: ", feat_class)
    print("Text: ", feat.text)
    print()
    print("Content: ", feat)

Type:  NavigableString
Name Tag:  None
Text:  20% 

Content:  20% 
Type:  NavigableString
Name Tag:  None
Text:   Hand

Content:   Hand
Type:  Tag
Name Tag:  br
Text:  

Content:  <br/>


In [21]:
block = blocks[5]
rows = block.find_all("div", class_="pi-item", recursive=False)
row = rows[3]
label = row.find("h3").text
item = row.find("div", class_="pi-data-value")

extract_item_vals(item, "text")

NameError: name 'blocks' is not defined

In [24]:
item_types = {
    "text": [
        "Introduced in", 
        "Required Home City Card", 
        "Hit points", 
        "Speed", 
        "Line of Sight", 
        "XP train bounty", 
        "XP kill bounty", 
        "Range", 
        "Rate of Fire", 
    ],
    "list": ["Type", "Civilization(s)", "Age", "Trained at"],
    "dict": ["Cost", "Train time", "Resistance", "Damage", "Bonus damage"],
}
unit_data = extract_unit_data(infobox, item_types)

Information for could not be extracted in dict format. Present values:  ['Same as ranged attack']


In [25]:
import json

print(json.dumps(unit_data, indent=4))

{
    "Information": {
        "Introduced in": "Age of Empires III",
        "Type": [
            "Cavalry",
            "Light ranged cavalry",
            "Ranged cavalry",
            "Gunpowder cavalry"
        ],
        "Civilization(s)": [
            "Germans",
            "John Black's Mercenaries (Act II: Ice)",
            "South Africa"
        ],
        "Age": [
            "Fortress Age",
            "Revolution (South Africa)"
        ],
        "Required Home City Card": "\"Wagon Warfare\" (South Africa)"
    },
    "Training": {
        "Trained at": [
            "Stable",
            "Fort",
            "Galleon",
            "Fluyt (South Africa)",
            "Steamer (Gran Colombia)"
        ],
        "Cost": {
            "Food": "150",
            "Coin": "150",
            "Population": "3"
        },
        "Train time": {
            "seconds": "60"
        }
    },
    "Statistics": {
        "Hit points": "500",
        "Resistance": {
            "Han